In [1]:
#import the libraries

import os
import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from transformers.models.roberta.modeling_roberta import RobertaForSequenceClassification, RobertaLayer
from transformers import TrainingArguments
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.activations import ACT2FN
import random

In [2]:
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging
#importing the dataset
from datasets import load_dataset
raw_datasets  = load_dataset("glue", 'mrpc')

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification


model_name = "FacebookAI/roberta-base"
# Load the configuration of the pre-trained model
config = AutoConfig.from_pretrained(model_name)

config.hidden_dropout_prob=0.0
config.attention_probs_dropout_prob=0.00
# Load the tokenizer for the pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding


tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

col_to_delete = ['sentence1','sentence2']

def preprocessing_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True,max_length=512)

tokenized_datasets = raw_datasets.map(preprocessing_function, batched=True, remove_columns=col_to_delete)

tokenized_datasets.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import RobertaForSequenceClassification
# Load a pre-trained Roberta model for sequence classification
model = RobertaForSequenceClassification.from_pretrained(
    model_name,

)

In [ ]:

# Define LoRA configuration parameters
lora_alpha = 8 
lora_dropout = 0.1 
lora_rank = 4 
# Initialize the LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        
        "query",
        "key",
        "value",
        "dense",
    ]
)
# Apply the LoRA configuration to the model
model = get_peft_model(model, peft_config)

In [ ]:
model = model.to('cuda')

In [34]:
import evaluate
import numpy as np
from sklearn import metrics
import torch
import numpy as np

def compute_metrics(eval_pred):


    logits, labels = eval_pred # eval_pred is the tuple of predictions and labels returned by the model
    predictions = np.argmax(logits, axis=-1)
    
    precision = metrics.precision_score(labels, predictions, average="macro")
    recall = metrics.recall_score(labels, predictions, average="macro")
    f1 = metrics.f1_score(labels, predictions, average="macro")
    accuracy = metrics.accuracy_score(labels, predictions)
    
    return {"precision": precision, "recall": recall, "f1-score": f1, 'accuracy': accuracy}

In [35]:
from transformers import TrainingArguments, Trainer



import time
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='qnli_dir',
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.00,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=10000000,
    logging_steps=100,
   
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",  
    warmup_steps=500,
)

trainer = Trainer(
    model=custom_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],

    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/home/s/sami20/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Train the model
trainer.train()

/home/s/sami20/miniconda3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
100,0.643700,0.604415,0.341912,0.500000,0.406114,0.683824
200,0.613300,0.625649,0.680556,0.523839,0.464145,0.693627
300,0.579700,0.559509,0.693241,0.545595,0.508360,0.703431
400,0.554400,0.505479,0.726134,0.701134,0.710140,0.762255
500,0.504800,0.450658,0.745770,0.784154,0.738225,0.747549
600,0.461500,0.398628,0.827273,0.756064,0.777140,0.825980
700,0.407200,0.395059,0.832128,0.807619,0.817894,0.848039


/home/s/sami20/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/s/sami20/miniconda3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/s/sami20/miniconda3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/s/sami20/miniconda3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will inst

TrainOutput(global_step=770, training_loss=0.5246252010395, metrics={'train_runtime': 438.0878, 'train_samples_per_second': 83.728, 'train_steps_per_second': 1.758, 'total_flos': 1569420106438080.0, 'train_loss': 0.5246252010395, 'epoch': 10.0})